In [85]:
import cv2
import numpy as np
import pandas as pd
import math

In [86]:
# Read the CSV file
df = pd.read_csv('../Data/2023-Cone/FullData.csv')
df

,Image,Center_X,Center_Y,Width,Height,saharAngle,x_position,y_position,angle
0,0,42,26,84,53,348.055823,-2.417500,3.22281,0.0
1,1,37,26,74,53,345.963757,-2.417500,3.22281,10.0
2,2,32,26,64,53,325.561011,-2.417500,3.22281,20.0
3,3,34,27,69,54,329.931417,-2.417500,3.22281,30.0
4,4,36,27,73,54,330.572544,-2.417500,3.22281,40.0
...,...,...,...,...,...,...,...,...,...
28363,28363,891,517,473,405,122.371778,0.142662,0.32281,310.0
28364,28364,902,523,500,394,12.619322,0.142662,0.32281,320.0
28365,28365,912,530,517,380,15.255119,0.142662,0.32281,330.0
28366,28366,922,537,516,365,19.932537,0.142662,0.32281,340.0


When a new image is captured we need to run the same algorithm we ran to create the data

In [87]:
# Detect game pice by color
# And create a rect around the detection
def find_bounding_rect(image, lower_color, upper_color):
    """    Detects the largest contour of a specified color in the image and returns its bounding rectangle.
    Args:
        image (numpy.ndarray): The input image in which to find the contour.
        lower_color (tuple): The lower bound of the color in HSV format.
        upper_color (tuple): The upper bound of the color in HSV format.
    Returns:
        tuple: A tuple containing the coordinates (x, y) and dimensions (width, height) of the bounding rectangle.
    """
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Combine masks to get only red pixels
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Threshold the mask to create a binary image
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Define the kernel for the morphological operation
    # ChatGPT gave me this part of the code, I have no idea how it works
    kernel = np.ones((5, 5), np.uint8)

    # Perform morphological opening to remove noise
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Perform morphological closing to close small holes
    # This really helps to cleanup the mask
    processed_img = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(processed_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for cont in contours:
        x, y, w, h = cv2.boundingRect(cont)
        cv2.drawContours(image, cont, -1, (0, 255, 0), 3)
        #cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)

    contour = max(contours, key=cv2.contourArea)
    
    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    cv2.drawContours(image, contour, -1, (0, 255, 0), 3)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)

    return (x, y, w, h), contour


def saharAlgorithm(contour, img):
    # Compute the center of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Find the point on the contour that is furthest from the center
    # It is almost always the cone top point
    # IMPORTANT: If the cone top is not visible in the image you will get undefined behavior
    distances = [np.linalg.norm(np.array([cX, cY]) - point[0]) for point in contour]
    max_distance_index = np.argmax(distances)
    furthest_point = tuple(contour[max_distance_index][0])

    X = furthest_point[0]
    Y = furthest_point[1]

    p1 = (cX, img.shape[0] - cY)
    p2 = (X, img.shape[0] - Y)

    # Difference in x coordinates
    dx = p2[0] - p1[0]

    # Difference in y coordinates
    dy = p2[1] - p1[1]

    # Angle between p1 and p2 in radians
    theta = math.atan2(dy, dx)

    angle = theta * 180 / math.pi

    if (dy < 0):
        angle = 360 + angle
    
    # Visualize
    cv2.line(img, (cX, cY), furthest_point, (255, 255, 255), 2)

    return angle

Find similar data points inside our data base to find the real world position

In [88]:

def angle_close(a, b, atol):
    # Compute the minimal difference between angles (in degrees)
    diff = np.abs((a - b + 180) % 360 - 180)
    return diff <= atol

def find_matching_rows(df, target, start_tol=2, angle_start_tol=10, max_tol=15, step=2):
    """
    Find rows in the dataframe that match the target values within a dynamically
    increasing tolerance. If no rows are found for a very small tolerance, the
    tolerance will be increased until at least one row is found or the maximum
    tolerance is reached.

    Parameters:
        df (pd.DataFrame): DataFrame with the columns.
        target (dict): Target values for each column.
        start_tol (int): Starting tolerance measured in pixels.
        max_tol (int): Maximum allowed tolerance measured in pixels.
        step (int): Increment to increase tolerance on each iteration measured in pixels.

    Returns:
        filtered_df (pd.DataFrame): DataFrame with matching rows.
        used_tol (int): Tolerance at which the matching rows were found.
    """
    
    tolerance = start_tol
    angle_tolerance = angle_start_tol
    while tolerance <= max_tol:
        # Create mask using np.isclose for each column
        mask = (
            np.isclose(df['Center_X'], target['Center_X'], atol=tolerance) &
            np.isclose(df['Center_Y'], target['Center_Y'], atol=tolerance) &
            np.isclose(df['Width'], target['Width'], atol=tolerance) &
            np.isclose(df['Height'], target['Height'], atol=tolerance) &
            angle_close(df['saharAngle'], target['saharAngle'], angle_tolerance)
        )
        filtered_df = df[mask]
        
        # Check if any row is found
        if not filtered_df.empty:
            print(f"Found rows with tolerance: {tolerance}")
            return filtered_df, tolerance
        
        # Increase the tolerance and try again
        tolerance += step
        angle_tolerance += step

    # No rows found within maximum tolerance
    print("No rows found within the max tolerance.")
    return df.iloc[[]], tolerance  # Return an empty DataFrame

Main function running on the Raspberry-PI

In [89]:
targetRes = (1280, 720)  # Target resolution for the image

# Capture the image using the connected camera (For testing purposes load an image from disk)
image = cv2.imread("../TestingImages/2023-Cone/frame11.jpg")

# Resize the image to the target size
image = cv2.resize(image, targetRes)

# The low bounds, [h, s, v]
low_bounds = np.array([6, 140, 85])
# The top bounds, [h, s, v]
top_bounds = np.array([27, 255, 255])

# Detect the closest game piece in the image, using ml model or a simple color filter
rect, contour = find_bounding_rect(image, low_bounds, top_bounds)
angle = saharAlgorithm(contour, image)

cv2.imshow("Frame", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

x, y, w, h = rect

target = {
    'Center_X': x + w // 2,
    'Center_Y': y + h // 2,
    'Width': w,
    'Height': h,
    'saharAngle': angle
}

# Assuming df is the pandas DataFrame
result_df, used_tolerance = find_matching_rows(df, target, start_tol=25, angle_start_tol=6, max_tol=40, step=3)
result_df

Found rows with tolerance: 25


,Image,Center_X,Center_Y,Width,Height,saharAngle,x_position,y_position,angle
22738,22738,371,114,148,143,315.000000,-0.533957,1.52281,220.0
23494,23494,342,126,156,154,312.825256,-0.552572,1.42281,220.0


For better accuracy we need to create more dense data with Blender (for example render an Image every 5 degrees and not every 10)

In [90]:
note_position = [result_df['x_position'].mean(), result_df['y_position'].mean(), result_df['angle'].mean()]
note_position

[-0.5432642474194344, 1.4728099999999984, 220.0]